<a href="https://colab.research.google.com/github/Mar3334/Patimetria/blob/main/Patimetria2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch==1.4.0 torchvision==0.5.0

     |████████████████████████████████| 753.4MB 15kB/s 
     |████████████████████████████████| 4.0MB 43.8MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [ ]:
#!git clone https://github.com/adambielski/siamese-triplet.git

Cloning into 'siamese-triplet'...
remote: Enumerating objects: 146, done.
remote: Total 146 (delta 0), reused 0 (delta 0), pack-reused 146
Receiving objects: 100% (146/146), 12.61 MiB | 33.54 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [ ]:
!cp -rf /content/drive/MyDrive/siamese-triplet /content/drive/MyDrive/siamese-triplet

cp: cannot copy a directory, '/content/drive/MyDrive/siamese-triplet', into itself, '/content/drive/MyDrive/siamese-triplet/siamese-triplet'


In [ ]:
import torch
from torchvision import models

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/siamese-triplet')

In [ ]:
from networks import VGGEmbeddingNet, EmbeddingNet, TripletNet
vgg_model = VGGEmbeddingNet()
emb_net = EmbeddingNet()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


In [ ]:
vgg_model_pretrained = VGGEmbeddingNet()

In [ ]:
from duckies_dataset import DuckieDataset

In [ ]:
from torchvision import transforms
from duckies_dataset import Rescale
trans = transforms.Compose([Rescale(224),
                            transforms.ToTensor()])

duckie_dataset = DuckieDataset('/content/drive/MyDrive/train', transform= trans)

In [ ]:
from datasets import TripletMNIST
triplet_dataset = TripletMNIST(duckie_dataset)

<class 'torch.Tensor'>


In [ ]:
from trainer import fit, train_epoch, test_epoch
from datasets import TripletMNIST
train_dataset = DuckieDataset("/content/drive/MyDrive/train", transform= trans)
test_dataset = DuckieDataset("/content/drive/MyDrive/train", train=False, transform= trans)

In [ ]:
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable
from losses import TripletLoss
import numpy as np
cuda = torch.cuda.is_available()
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
# Set up data loaders

triplet_train_dataset = TripletMNIST(train_dataset) # Returns triplets of images
triplet_test_dataset = TripletMNIST(test_dataset)
batch_size = 32
kwargs = {'num_workers': 8, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)

triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

# Set up the network and training parameters

margin = 1.
embedding_net = EmbeddingNet()
model = TripletNet(vgg_model)
if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 10
log_interval = 500

<class 'torch.Tensor'>


In [ ]:
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train: [0/1080 (0%)]	Loss: 0.808462
Epoch: 1/10. Train set: Average loss: 0.9710
Epoch: 1/10. Validation set: Average loss: 0.9462
Train: [0/1080 (0%)]	Loss: 0.952875
Epoch: 2/10. Train set: Average loss: 0.7757
Epoch: 2/10. Validation set: Average loss: 0.4199
Train: [0/1080 (0%)]	Loss: 0.707611
Epoch: 3/10. Train set: Average loss: 0.5420
Epoch: 3/10. Validation set: Average loss: 0.5285
Train: [0/1080 (0%)]	Loss: 0.696592
Epoch: 4/10. Train set: Average loss: 0.5330
Epoch: 4/10. Validation set: Average loss: 0.4279
Train: [0/1080 (0%)]	Loss: 0.552867
Epoch: 5/10. Train set: Average loss: 0.5257
Epoch: 5/10. Validation set: Average loss: 0.5201
Train: [0/1080 (0%)]	Loss: 0.422226
Epoch: 6/10. Train set: Average loss: 0.5120
Epoch: 6/10. Validation set: Average loss: 0.4105
Train: [0/1080 (0%)]	Loss: 0.343924
Epoch: 7/10. Train set: Average loss: 0.4788
Epoch: 7/10. Validation set: Average loss: 0.4042
Train: [0/1080 (0%)]	Loss: 0.511406
Epoch: 8/10. Train set: Average loss: 0.3987
Ep

In [ ]:
# Define functions to extract embeddings using diferent datasets
def extract_embeddings(dataset, model, dims = 1024, opt = None):
    with torch.no_grad():
        model.eval()
        embeddings = np.zeros((len(dataset), dims))
        labels = list()
        for k, it_ in enumerate(dataset):
            images = it_[0].unsqueeze_(0)
            images = images.cuda()
            target = it_[1]
            if opt is None:
                aux = model.get_embedding(images).data.cpu().numpy()
            else:
                aux = model.get_embedding(images, opt).data.cpu().numpy()
            embeddings[k] = aux.reshape(1, dims)
            labels.append(target)
    return embeddings, labels


In [ ]:
embeddings, labels = extract_embeddings(test_dataset, vgg_model, dims = 512)

In [ ]:
'''
emb1 = embeddings[-1,:]
print(emb1.shape)
for i in range(embeddings.shape[0]):
    emb2 = embeddings[i,:]
    dist = np.sum((emb2 - emb1)**2)
    print(dist, labels[i])
'''

(512,)
1.6502837951017366 tensor(228.)
1.8217226019836312 tensor(228.)
1.970004959507878 tensor(229.)
0.9628877005756225 tensor(229.)
1.4609063977398042 tensor(229.)
1.4613784141557664 tensor(229.)
1.8781259397282728 tensor(230.)
1.2236530895505244 tensor(230.)
1.798596555973646 tensor(230.)
1.7364517478788835 tensor(230.)
1.8507907801226602 tensor(231.)
1.6892059898605782 tensor(231.)
1.8354687488457837 tensor(231.)
1.8880668121763542 tensor(232.)
1.9540483111439266 tensor(232.)
1.7913181982568307 tensor(232.)
1.9540613625960606 tensor(232.)
1.9540527338551892 tensor(232.)
1.3281735956217426 tensor(233.)
1.6255817298604536 tensor(233.)
1.6880882394897996 tensor(233.)
1.9366967344248587 tensor(233.)
0.2859212286321504 tensor(234.)
0.12068297838695373 tensor(234.)
0.5243246085165081 tensor(234.)
0.08158551760496088 tensor(234.)
1.9521709227504003 tensor(235.)
1.1640232504269765 tensor(235.)
1.5296535613548616 tensor(235.)
1.778552412895722 tensor(235.)
1.954044980606761 tensor(236.)
1.9

In [ ]:
embeddings1, labels1 = extract_embeddings(test_dataset, vgg_model_pretrained.cuda(), dims = 512)

In [ ]:
'''
emb1 = embeddings1[-1,:]
print(emb1.shape)
for i in range(embeddings1.shape[0]):
    emb2 = embeddings1[i,:]
    dist = np.sum((emb2 - emb1)**2)
    print(dist, labels1[i])
'''

(512,)
0.630583970944256 tensor(228.)
0.5489316875561194 tensor(228.)
0.5401648445001825 tensor(229.)
0.579095291802384 tensor(229.)
0.3966684128232094 tensor(229.)
0.4195389272663902 tensor(229.)
0.43249176994107597 tensor(230.)
0.519822967579026 tensor(230.)
0.35402580539168627 tensor(230.)
0.36921421131803506 tensor(230.)
0.5761297998356337 tensor(231.)
0.5503901105303631 tensor(231.)
0.5043408889681327 tensor(231.)
0.8570932861951437 tensor(232.)
0.6134906876702113 tensor(232.)
0.8036465265789257 tensor(232.)
0.5703406317007375 tensor(232.)
0.5456899517690402 tensor(232.)
0.8322622080529798 tensor(233.)
0.7355504444094727 tensor(233.)
0.6496333637506815 tensor(233.)
0.6804755234783255 tensor(233.)
0.6332883895895459 tensor(234.)
0.44739782683984713 tensor(234.)
0.5729899107555607 tensor(234.)
0.46852782932405873 tensor(234.)
1.2457919125829382 tensor(235.)
0.9145934490328036 tensor(235.)
0.89636740990823 tensor(235.)
0.7484997093079961 tensor(235.)
0.865921473643684 tensor(236.)
0.

In [ ]:
from PIL import Image
def n_closest_images(image, n=5):
  img = Image.open(image)
  img = trans(img)
  emb_img = vgg_model(img.unsqueeze(0).cuda())
  embedding_img = emb_img.data.cpu().numpy()
  list_dist = []
  list_labels = []
  neat_list = []
  label_ubication_list = []
  for i in range(embeddings.shape[0]):
    emb2 = embeddings[i,:]
    dist = np.sum((emb2 - embedding_img)**2)
    list_dist.append(dist)
  neat_list[:] = list_dist
  neat_list.sort()
  for i in range(n):
    position = list_dist.index(neat_list[i])
    label_ubication = position - labels.index(labels[position])
    label_ubication_list.append(label_ubication)
    list_labels.append(labels[position])
  print (neat_list[:n], list_labels, label_ubication_list)






In [ ]:
n_closest_images("/content/drive/MyDrive/Patos/Lucas/PatoLucas.jpg")

[0.01503039433636619, 0.015604701639392727, 0.019240022737123247, 0.020296860477561466, 0.022060672812869535] [tensor(281.), tensor(273.), tensor(281.), tensor(251.), tensor(273.)] [3, 1, 1, 1, 0]


In [ ]:
'''
from PIL import Image

pato_lucas = Image.open("/content/drive/MyDrive/Patos/Lucas/PatoLucas.jpg")
pato_lucas = trans(pato_lucas)
emb_pato_lucas = vgg_model(pato_lucas.unsqueeze(0).cuda())
print (emb_pato_lucas.shape)
'''

'\nfrom PIL import Image\n\npato_lucas = Image.open("/content/drive/MyDrive/Patos/Lucas/PatoLucas.jpg")\npato_lucas = trans(pato_lucas)\nemb_pato_lucas = vgg_model(pato_lucas.unsqueeze(0).cuda())\nprint (emb_pato_lucas.shape)\n'

In [ ]:
'''
emb1 = emb_pato_lucas.data.cpu().numpy()
print(emb1.shape)
for i in range(embeddings.shape[0]):
    emb2 = embeddings[i,:]
    dist = np.sum((emb2 - emb1)**2)
    print(dist, labels[i])
'''

'\nemb1 = emb_pato_lucas.data.cpu().numpy()\nprint(emb1.shape)\nfor i in range(embeddings.shape[0]):\n    emb2 = embeddings[i,:]\n    dist = np.sum((emb2 - emb1)**2)\n    print(dist, labels[i])\n'

In [ ]:
#test_dataset.class_to_idx

In [ ]:
lista = [1,1,2,3]
lista.index(1)

0